In [2]:
# Importation des libraries/packages de base. 

import pandas as pd
import os
import numpy as np
import dask.dataframe as dd 
import warnings 
warnings.filterwarnings("ignore")



In [7]:
# Fichier avec les variables d'intéret : dernière version : 21/02 

# df_cct = pd.read_parquet("/Users/vanheuverswyn/Desktop/ProjetStatapp/df_all.parquet")

In [8]:
#creation de 2000 parquets (1 par user)
#for k in range (2000):
    #df=df_cct[df_cct['User']==k]
    #df.to_parquet('/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_all/df_all'+ str(k)+'.parquet')



In [24]:
from sklearn import preprocessing
#fonction qui retire les colonnes inutiles, qui cree la colonne 'Number_year_since_PIN_last_Changed' qui
    #qui cree des dummies a partir des variable categorielle et qui normalise les variables non-categorielles

def modification_data_frame(df_cct):
    
    #On retire les colonnes qui ne sont plus utiles
    df_cct =  df_cct.drop([  'day_of_week','Card', 'Month', 'Day', 'Hours'], axis = 1)
    
    
    #pour ne pas avoir de soucis lors de la concatenations des dataframe (df_cct et dummies), on reinitialise les index
    df_cct=df_cct.reset_index()    
    df_cct =  df_cct.drop(["index"], axis = 1)

    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    features_array = enc.fit_transform(df_cct[['Card Brand']]).toarray()
    features_labels = np.hstack(np.array(enc.categories_))
    df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels).reset_index()], axis = 1)

    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    features_array = enc.fit_transform(df_cct[['Use Chip']]).toarray()
    features_labels = np.hstack(np.array(enc.categories_))
    df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels).reset_index()], axis = 1)

    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    features_array = enc.fit_transform(df_cct[['Card Type']]).toarray()
    features_labels = np.hstack(np.array(enc.categories_))
    df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels)], axis = 1)

    df_cct =  df_cct.drop(["index","Card Brand",'Card Type'], axis = 1)
    
    
    
    #On centre et reduit les colonnes a normaliser suivantes:
    Normalizing_List=['Amount', 'Credit Limit', 'delta_t_s', 'delta_t_s_card',
       'amt/daily_income', 'daily_amount', 'nb_daily_declines_card',
       'nb_weekly_declines_card', 'nb_monthly_declines_card',
       'hr_nbt/last_30d_av_hr_nbt',
       'day_nbt/last_30d_av_day_nbt', 'last_3d_amt/nbt',
       'year_diff']
    df_cct_not_normalized=df_cct.copy()
    for i in Normalizing_List:
        if np.std(df_cct[i])!=0:
            df_cct[i] = (df_cct[i]-np.mean(df_cct[i]))/np.std(df_cct[i])
        else:
            df_cct[i] = (df_cct[i]-np.mean(df_cct[i])) 
    return df_cct
        
                
        
        
        
    

In [25]:
# Application de la fonction au 2000 parquet -> on obtient les parquets modifiés et les parquets avec
# les fraudes associees pour chaque utilisatuer dans un seul et même dossier.  
    
    
for k in range (2000):
    df=pd.read_parquet("/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_all/df_all"+str(k)+".parquet")
    
    df_modified=modification_data_frame(df)
    IsFraud=df_modified["Is Fraud?"].to_frame()
    df_modified=df_modified.drop(["Is Fraud?"],axis = 1)
    
    df_modified.to_parquet('/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_modified_all/df_all'+ str(k)+'.parquet')
    IsFraud.to_parquet('/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_modified_all/IsFraud'+ str(k)+'.parquet')
    


In [25]:
#on ouvre le parquet 200 pour tester
# df200=pd.read_parquet("/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_all/df_all200.parquet").head()
# df200.head()

In [24]:
#on ouvre le parquet 200 pour tester
# df200_modified=pd.read_parquet("/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_modified_all/df_all200.parquet")
# print(df200_modified.describe())

In [6]:
#fonction qui applique dbScann au dataFrame A finir

    
def dbScann (df_cct):
    X = df_cct.values
    X.astype(np.float64)
    #eps min_sample et n_jobs a déterminer  (fixer le meme pour tous ou automatiser une selection?)
    clustering=cluster.DBSCAN(eps=0.8, min_samples=5, n_jobs = -1)
    clustering.fit(X)
    
def PCS(df_cct):    
    pca = PCA()
    pca.fit(df_cct)
    df_cct.columns = df_cct.columns.astype("str")
    df_cct_pca = pca.transform(df_cct)
    df_cct_pca_df = pd.DataFrame({
    "Dim1" : df_cct_pca[:,0], 
    "Dim2" : df_cct_pca[:,1]})
    df_cct_pca_df["labels"] = clustering.labels_



In [ ]:

# df =pd.read_parquet("/home/onyxia/work/Databases/df_all"+str(i)+".parquet")

In [5]:
# Commençons par les 200 premiers Users : 

f = os.listdir("/home/onyxia/work/Datasets")
f = [v for v in f if "df_all" in v][:200]


In [38]:
# Création d'une liste de dataframe :

df_cct = [pd.read_parquet("/home/onyxia/work/Datasets/" + v) for v in f]

In [39]:
# On a déjà retiré les variables catégorielles : on retire aussi User : 
df_cct = [df_cct[i].drop("User",  axis = "columns") for i in range(len(df_cct))]

In [111]:
# Fonction pour supprimer les variables catégorielles : 

def reshapedf(df):
    unique_values = {v : df[v].unique() for v in df.columns}
    unique_values = {v : unique_values[v] for v in unique_values.keys() if len(unique_values[v]) <= 2}
    if "Credit Limit" in list(unique_values.keys()) : 
        del unique_values['Credit Limit']
    return df.drop(list(unique_values.keys()), axis = "columns")

In [112]:
#df_cct = [reshapedf(df) for df in df_cct]


In [19]:
from sklearn import cluster 
from collections import Counter

In [27]:
# Fonction pour implémenter DBSCAN sur un dataset et retourne le nbr de cluster : 

def DBSCAN1(df_cct, eps, min_sample):
    X = df_cct.values
    X.astype(np.float64)
    #eps min_sample et n_jobs a déterminer  (fixer le meme pour tous ou automatiser une selection?)
    clustering=cluster.DBSCAN(eps = eps, min_samples = min_sample, n_jobs = -1)
    clustering.fit(X)
    return Counter(clustering.labels_)

In [28]:
# DBSCAN + insérer le label dans une nouvelle variable pour chaque transaction. 

def DBSCAN2(df, eps, min_sample):
    X = df.values
    X.astype(np.float64)
    #eps min_sample et n_jobs a déterminer  (fixer le meme pour tous ou automatiser une selection?)
    clustering=cluster.DBSCAN(eps = eps, min_samples = min_sample, n_jobs = -1)
    clustering.fit(X)
    df.insert(1,"label", clustering.labels_)
    return df

In [ ]:
# Fonctions pour calculer la distance moyenne entre les points : trouver eps : 

def Euclid_distance(point1, point2):
    return np.linalg.norm(point1 - point2)
    
def Transac_distance(point1, set2):
    return np.mean([Euclid_distance(point1, point2) for point2 in set2])

def User_distance(User):
    return np.mean([Transac_distance(point1, User) for point1 in User])


In [23]:
# Choix de eps : 
User_distance(df_cct[0].values) # 4.069

4.069536775684035

In [34]:
# Test : trouver min_sample : idée : 0.002*df.shape[0] : regarder un peu le nbr de cluster et effectif : 
[DBSCAN1(df, 2.5, max(2,round(0.002*df.shape[0]))) for df in df_cct[:20]]
# 2.5 satisfaisant pour eps : regarder nbr de cluster et effectif 

[Counter({-1: 418, 0: 15042, 1: 272}),
 Counter({0: 47, -1: 7, 1: 2}),
 Counter({0: 19004, -1: 556, 1: 355}),
 Counter({0: 7306, -1: 265, 1: 174}),
 Counter({-1: 390, 0: 204, 1: 9715, 2: 28}),
 Counter({-1: 9, 0: 112}),
 Counter({0: 11, -1: 7, 1: 2, 2: 34, 3: 3}),
 Counter({-1: 48, 0: 1729, 1: 11, 2: 150, 3: 7, 4: 24, 5: 10}),
 Counter({0: 24966, -1: 622, 1: 554}),
 Counter({0: 2929, -1: 84, 1: 77, 2: 5, 3: 76}),
 Counter({0: 269, -1: 49, 1: 5, 2: 8, 3: 5, 4: 1413, 5: 17}),
 Counter({0: 9728, 1: 302, -1: 331, 2: 24}),
 Counter({-1: 507, 0: 21974, 1: 427}),
 Counter({0: 12863, -1: 292, 1: 423, 2: 1509}),
 Counter({0: 4449,
          1: 59,
          2: 204,
          -1: 212,
          3: 32,
          6: 23,
          4: 1007,
          5: 23,
          7: 18}),
 Counter({0: 41, -1: 236, 1: 10774, 2: 264}),
 Counter({0: 9140, -1: 248, 2: 227, 1: 603}),
 Counter({0: 16171, 1: 428, 3: 40, -1: 476, 2: 65}),
 Counter({0: 100, 1: 2, -1: 12, 2: 2, 3: 5, 4: 13, 5: 2, 6: 2}),
 Counter({-1: 197

In [40]:
# On affecte le numéro du cluster à chaque transaction : # On test sur les 

df_cct_clustered = [DBSCAN2(df,2.5, max(2,round(0.002*df.shape[0]))) for df in df_cct[:20]]

In [77]:
# On rassemble les trasactions par label : et on prend la moyenne :  


df_cct_clustered1 = [df.groupby("label").mean().reset_index() for df in df_cct_clustered]

In [73]:
df_cct_clustered1[0].head()

,label,Amount,Credit Limit,year_diff,delta_t_s,delta_t_s_card,amt/daily_income,daily_amount,nb_daily_declines_card,nb_weekly_declines_card,nb_monthly_declines_card,hr_nbt/last_30d_av_hr_nbt,day_nbt/last_30d_av_day_nbt,last_3d_amt/nbt
0,-1,1.701182,-0.554838,0.408695,0.220138,0.201935,1.701182,4.028080,1.182968,0.991471,0.506329,1.047170,0.730244,2.986989
1,0,-0.047251,0.010827,-0.010274,-0.004693,-0.003679,-0.047251,-0.114810,-0.148595,-0.062590,-0.029429,-0.039514,-0.026954,-0.080199
2,1,-0.001256,0.253896,-0.059878,-0.078760,-0.106897,-0.001256,0.158949,6.399559,1.937671,0.849361,0.575938,0.368395,-0.155173


In [50]:
# On rassemble le tout : + DBSCAN : 

df_final = pd.concat(df_cct_clustered1).reset_index(drop = True).drop("label", axis = "columns")

In [51]:
User_distance(df_final.values)

5.929339307877806

In [60]:
DBSCAN1(df_final,2.5,max(2,round(0.002*df_final.shape[0])))

Counter({0: 15, 1: 27, 2: 18, 3: 4, -1: 18, 4: 6, 5: 2})

In [58]:
df_final1 = DBSCAN2(df_final,2.5,max(2,round(0.002*df_final.shape[0])))

In [59]:
df_final1.head()

,Amount,label,Credit Limit,year_diff,delta_t_s,delta_t_s_card,amt/daily_income,daily_amount,nb_daily_declines_card,nb_weekly_declines_card,nb_monthly_declines_card,hr_nbt/last_30d_av_hr_nbt,day_nbt/last_30d_av_day_nbt,last_3d_amt/nbt
0,1.701182,0,-0.554838,0.408695,0.220138,0.201935,1.701182,4.028080,1.182968,0.991471,0.506329,1.047170,0.730244,2.986989
1,-0.047251,1,0.010827,-0.010274,-0.004693,-0.003679,-0.047251,-0.114810,-0.148595,-0.062590,-0.029429,-0.039514,-0.026954,-0.080199
2,-0.001256,2,0.253896,-0.059878,-0.078760,-0.106897,-0.001256,0.158949,6.399559,1.937671,0.849361,0.575938,0.368395,-0.155173
3,0.866996,1,0.000000,0.000000,0.234966,0.234966,0.866996,0.886391,1.347151,1.072222,0.219382,-0.266760,-0.013971,0.518652
4,-0.044472,1,0.000000,0.000000,0.002368,0.002368,-0.044472,-0.183116,-0.192450,-0.135125,-0.066904,-0.084747,-0.072587,-0.108510


In [78]:
# On revient sur le dataset de base : 
# Créer la variable label_global :

# On commence par remettre le USER : 

[df.insert(1,"User", list(np.ones((5,), dtype=int))) for i,df in enumerate(df_cct_clustered1)]

ValueError: Length of values (0) does not match length of index (3)

In [70]:
print(df_final_ut[1])

None
